In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


## Import and load dataframe

In [2]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix 

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [3]:
train_file = "/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv"
test_file = "/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv"
traindf = pd.read_csv(train_file)
testdf = pd.read_csv(test_file)

In [137]:
traindf.head()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase,Made_Purchase
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1.0,9.0,3.0,Returning_Visitor,Not Specified,Deny,Not Specified,Other,0.0,False
1,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Female,Deny,Others,Married,0.0,False
2,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,...,3.0,1.0,4.0,Returning_Visitor,Female,ALL,Others,Married,1.0,False
3,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.4,...,4.0,3.0,3.0,Returning_Visitor,Male,ALL,Diploma,Single,0.0,False
4,1.0,0.0,0.0,0.0,0.0,0.000000,0.20,0.20,0.0,0.0,...,2.0,1.0,5.0,Returning_Visitor,Male,Deny,Diploma,Other,1.0,False


In [138]:
testdf.head()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,OS,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,1.0,1.0,1.0,1.0,Returning_Visitor,Not Specified,Deny,Not Specified,Single,0.0
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,...,2.0,2.0,1.0,2.0,Returning_Visitor,Not Specified,ALL,Graduate,Married,0.0
2,0.0,0.0,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,...,2.0,2.0,1.0,3.0,Returning_Visitor,Female,Required,Diploma,Other,0.0
3,0.0,0.0,0.0,0.0,2.0,37.000000,0.000000,0.100000,0.0,0.8,...,2.0,2.0,2.0,3.0,Returning_Visitor,Not Specified,Required,Graduate,Other,0.0
4,0.0,0.0,0.0,0.0,16.0,407.750000,0.018750,0.025833,0.0,0.4,...,1.0,1.0,4.0,3.0,Returning_Visitor,Female,Deny,Others,Other,0.0


In [139]:
categtr = traindf.select_dtypes(exclude=['float64'])
categtr = categtr.drop(columns=['Made_Purchase'])
categtr_og = categtr.copy(deep=True)
ytr = traindf[['Made_Purchase']].copy(deep = True)

categte = testdf.select_dtypes(exclude=['float64'])
categte_og = categte.copy(deep=True)

print('Categorical fields -->',categtr.columns)
categtr

Categorical fields --> Index(['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies Setting',
       'Education', 'Marital Status'],
      dtype='object')


,Month_SeasonalPurchase,CustomerType,Gender,Cookies Setting,Education,Marital Status
0,Feb,Returning_Visitor,Not Specified,Deny,Not Specified,Other
1,Feb,Returning_Visitor,Female,Deny,Others,Married
2,Feb,Returning_Visitor,Female,ALL,Others,Married
3,Feb,Returning_Visitor,Male,ALL,Diploma,Single
4,Feb,Returning_Visitor,Male,Deny,Diploma,Other
...,...,...,...,...,...,...
14726,Nov,Returning_Visitor,Female,Deny,NaN,Married
14727,May,Returning_Visitor,Not Specified,ALL,Diploma,Married
14728,Aug,Returning_Visitor,Not Specified,Required,Others,Single
14729,Mar,Returning_Visitor,Not Specified,Required,Not Specified,Married


In [140]:
numertr = traindf.select_dtypes(['float64'])
numertr_og = numertr.copy(deep=True)
numerte = testdf.select_dtypes(['float64'])
numerte_og = numerte.copy(deep=True)

print('Numerical fields -->',numertr.columns)
numertr.head()

Numerical fields --> Index(['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
       'GoogleMetric:Page Values', 'SeasonalPurchase', 'OS', 'SearchEngine',
       'Zone', 'Type of Traffic', 'WeekendPurchase'],
      dtype='object')


,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,OS,SearchEngine,Zone,Type of Traffic,WeekendPurchase
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,4.0,1.0,9.0,3.0,0.0
1,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,3.0,2.0,2.0,4.0,0.0
2,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,3.0,3.0,1.0,4.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.4,2.0,4.0,3.0,3.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.000000,0.20,0.20,0.0,0.0,1.0,2.0,1.0,5.0,1.0


In [141]:
def null_present(df):
    for attr in df.columns:
        nullPresent = False
        for val in df[attr]:
            if type(val) is str:
                if val=='nan':
                    nullPresent = True
                    break
                else:
                    continue
            if np.isnan(val): #val == np.nan:
                nullPresent = True
                break
        print(attr,' -- ',nullPresent)
    print('-----------------------------')

In [142]:
def unique_vals(df):
    for attr in df.columns:
        print(attr,' unique vals \t---\t',set(df[attr]))
    print('---------------------------------------------')

In [143]:
print('          Numerical Train')
null_present(numertr)
print('          Numerical Test')
null_present(numerte)

          Numerical Train
HomePage  --  True
HomePage_Duration  --  True
LandingPage  --  True
LandingPage_Duration  --  True
ProductDescriptionPage  --  True
ProductDescriptionPage_Duration  --  True
GoogleMetric:Bounce Rates  --  True
GoogleMetric:Exit Rates  --  True
GoogleMetric:Page Values  --  True
SeasonalPurchase  --  True
OS  --  True
SearchEngine  --  True
Zone  --  True
Type of Traffic  --  True
WeekendPurchase  --  True
-----------------------------
          Numerical Test
HomePage  --  True
HomePage_Duration  --  True
LandingPage  --  True
LandingPage_Duration  --  True
ProductDescriptionPage  --  True
ProductDescriptionPage_Duration  --  True
GoogleMetric:Bounce Rates  --  True
GoogleMetric:Exit Rates  --  True
GoogleMetric:Page Values  --  True
SeasonalPurchase  --  True
OS  --  True
SearchEngine  --  True
Zone  --  True
Type of Traffic  --  True
WeekendPurchase  --  True
-----------------------------


In [144]:
print('          Categorical Train')
null_present(categtr)
print('          Categorical Test')
null_present(categte)

          Categorical Train
Month_SeasonalPurchase  --  True
CustomerType  --  True
Gender  --  True
Cookies Setting  --  True
Education  --  True
Marital Status  --  True
-----------------------------
          Categorical Test
Month_SeasonalPurchase  --  True
CustomerType  --  True
Gender  --  True
Cookies Setting  --  True
Education  --  True
Marital Status  --  True
-----------------------------


In [145]:
ytrain = ytr.iloc[:,0]
print(set(ytrain))
# ytr = pd.DataFrame(LabelEncoder().fit_transform(ytr), columns=ytr.columns)
ytrain = LabelEncoder().fit_transform(ytrain)
print(set(ytrain))

{False, True}
{0, 1}


# Imputing Numerical Data

In [146]:
k = int(np.sqrt(numertr.shape[0]))

numer_temp = None

knn_imputer = KNNImputer(n_neighbors=k)
knn_imputer.fit(numertr)
numer_temp = pd.DataFrame(knn_imputer.transform(numertr) , columns = numertr_og.columns)
numer_temp

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,OS,SearchEngine,Zone,Type of Traffic,WeekendPurchase
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,4.0,1.0,9.0,3.0,0.0
1,0.0,0.000000,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,3.0,2.0,2.0,4.0,0.0
2,0.0,0.000000,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,3.0,3.0,1.0,4.0,1.0
3,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.4,2.0,4.0,3.0,3.0,0.0
4,1.0,0.000000,0.0,0.0,0.0,0.000000,0.200000,0.200000,0.0,0.0,1.0,2.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14726,1.0,4.000000,0.0,0.0,39.0,983.138889,0.015385,0.017599,0.0,0.0,3.0,2.0,6.0,3.0,0.0
14727,8.0,117.023809,2.0,57.0,11.0,252.892857,0.000000,0.011078,0.0,0.0,2.0,2.0,2.0,4.0,0.0
14728,2.0,75.600000,2.0,652.8,10.0,1143.666667,0.000000,0.023333,0.0,0.0,2.0,2.0,4.0,2.0,0.0
14729,0.0,0.000000,0.0,0.0,6.0,1057.000000,0.000000,0.033333,0.0,0.0,2.0,4.0,4.0,1.0,0.0


In [147]:
null_present(numer_temp)

HomePage  --  False
HomePage_Duration  --  False
LandingPage  --  False
LandingPage_Duration  --  False
ProductDescriptionPage  --  False
ProductDescriptionPage_Duration  --  False
GoogleMetric:Bounce Rates  --  False
GoogleMetric:Exit Rates  --  False
GoogleMetric:Page Values  --  False
SeasonalPurchase  --  False
OS  --  False
SearchEngine  --  False
Zone  --  False
Type of Traffic  --  False
WeekendPurchase  --  False
-----------------------------


In [148]:
numertr = numer_temp.copy(deep=True)
numerte = pd.DataFrame(knn_imputer.transform(numerte) , columns = numerte_og.columns)
null_present(numerte)

HomePage  --  False
HomePage_Duration  --  False
LandingPage  --  False
LandingPage_Duration  --  False
ProductDescriptionPage  --  False
ProductDescriptionPage_Duration  --  False
GoogleMetric:Bounce Rates  --  False
GoogleMetric:Exit Rates  --  False
GoogleMetric:Page Values  --  False
SeasonalPurchase  --  False
OS  --  False
SearchEngine  --  False
Zone  --  False
Type of Traffic  --  False
WeekendPurchase  --  False
-----------------------------


# Imputing Categorical Data - Manual + SimpleImputer

In [149]:
unique_vals(categtr)

Month_SeasonalPurchase  unique vals 	---	 {nan, 'Oct', 'Mar', 'Nov', 'Sep', 'Aug', 'Dec', 'May', 'Feb', 'Jul', 'June'}
CustomerType  unique vals 	---	 {nan, 'Returning_Visitor', 'Other', 'New_Visitor'}
Gender  unique vals 	---	 {nan, 'Not Specified', 'Female', 'Male'}
Cookies Setting  unique vals 	---	 {'Deny', nan, 'Required', 'ALL'}
Education  unique vals 	---	 {nan, 'Diploma', 'Graduate', 'Not Specified', 'Others'}
Marital Status  unique vals 	---	 {'Single', nan, 'Other', 'Married'}
---------------------------------------------


## Manual Imputing
* Gender nan='Not Specified'
* Education nan='Not Specified'

In [150]:
manual = {'Gender' : 'Not Specified' , 'Education' : 'Not Specified'}

for attr in manual :
    for ind in range(len(categtr[attr])):
        val = categtr[attr][ind]
        
        if type(val) is str:
            if val == 'nan':
                categtr[attr][ind] = manual[attr]
        elif np.isnan(val):
            categtr[attr][ind] = manual[attr]
            
    for ind in range(len(categte[attr])):
        val = categte[attr][ind]
        
        if type(val) is str:
            if val == 'nan':
                categte[attr][ind] = manual[attr]
        elif np.isnan(val):
            categte[attr][ind] = manual[attr]

# Checking NaN
print('           Categorical Train')
unique_vals(categtr)
print('           Categorical Test')
unique_vals(categte)

           Categorical Train
Month_SeasonalPurchase  unique vals 	---	 {nan, 'Oct', 'Mar', 'Nov', 'Sep', 'Aug', 'Dec', 'May', 'Feb', 'Jul', 'June'}
CustomerType  unique vals 	---	 {nan, 'Returning_Visitor', 'Other', 'New_Visitor'}
Gender  unique vals 	---	 {'Not Specified', 'Female', 'Male'}
Cookies Setting  unique vals 	---	 {'Deny', nan, 'Required', 'ALL'}
Education  unique vals 	---	 {'Not Specified', 'Graduate', 'Others', 'Diploma'}
Marital Status  unique vals 	---	 {'Single', nan, 'Other', 'Married'}
---------------------------------------------
           Categorical Test
Month_SeasonalPurchase  unique vals 	---	 {nan, 'Oct', 'Mar', 'Nov', 'Sep', 'Aug', 'Dec', 'May', 'Feb', 'Jul', 'June'}
CustomerType  unique vals 	---	 {nan, 'Returning_Visitor', 'Other', 'New_Visitor'}
Gender  unique vals 	---	 {'Not Specified', 'Female', 'Male'}
Cookies Setting  unique vals 	---	 {'Deny', nan, 'Required', 'ALL'}
Education  unique vals 	---	 {'Not Specified', 'Graduate', 'Others', 'Diploma'}
Mar

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Simple Imputer

In [151]:
categ_si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categ_si.fit(categtr)

categtr = pd.DataFrame(categ_si.transform(categtr), columns=categtr.columns)
print('           Categorical Train')
unique_vals(categtr)

categte = pd.DataFrame(categ_si.transform(categte), columns=categte.columns)
print('           Categorical Test')
unique_vals(categte)

           Categorical Train
Month_SeasonalPurchase  unique vals 	---	 {'Oct', 'Mar', 'Nov', 'Sep', 'Aug', 'Dec', 'May', 'Feb', 'Jul', 'June'}
CustomerType  unique vals 	---	 {'Returning_Visitor', 'Other', 'New_Visitor'}
Gender  unique vals 	---	 {'Not Specified', 'Female', 'Male'}
Cookies Setting  unique vals 	---	 {'Deny', 'Required', 'ALL'}
Education  unique vals 	---	 {'Not Specified', 'Graduate', 'Others', 'Diploma'}
Marital Status  unique vals 	---	 {'Single', 'Other', 'Married'}
---------------------------------------------
           Categorical Test
Month_SeasonalPurchase  unique vals 	---	 {'Oct', 'Mar', 'Nov', 'Sep', 'Aug', 'Dec', 'May', 'Feb', 'Jul', 'June'}
CustomerType  unique vals 	---	 {'Returning_Visitor', 'Other', 'New_Visitor'}
Gender  unique vals 	---	 {'Not Specified', 'Female', 'Male'}
Cookies Setting  unique vals 	---	 {'Deny', 'Required', 'ALL'}
Education  unique vals 	---	 {'Not Specified', 'Graduate', 'Others', 'Diploma'}
Marital Status  unique vals 	---	 {'Ot

# Transform Categorical - OneHotEncoder

In [152]:
ohe = OneHotEncoder()
ohe.fit(categtr)
categtr = pd.DataFrame(ohe.transform(categtr).toarray())
categte = pd.DataFrame(ohe.transform(categte).toarray())

categtr.shape , categte.shape

((14731, 26), (6599, 26))

In [153]:
categtr.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


# Scaling numerical data - Standard scaler

In [154]:
std_sc = StandardScaler()
std_sc.fit(numertr)
numertr = pd.DataFrame(std_sc.transform(numertr) , columns = numertr.columns)
numerte = pd.DataFrame(std_sc.transform(numerte) , columns = numerte.columns)

# Concatenate numerical and categorical data

In [155]:
Xtrain = pd.concat([numertr,categtr] , axis=1)
Xtest = pd.concat([numerte,categte] , axis=1)

# Data Type Fixing

In [156]:
Xtrain.columns = Xtrain.columns.astype(str)
Xtest.columns = Xtest.columns.astype(str)

# Feature Extraction

In [157]:
selector = SelectKBest(mutual_info_classif, k=7)

In [158]:
selector.fit(Xtrain, ytrain)
Xtrain = pd.DataFrame(selector.transform(Xtrain) , columns=Xtrain.columns[selector.get_support(indices=True)])
Xtest = pd.DataFrame(selector.transform(Xtest) , columns=Xtest.columns[selector.get_support(indices=True)])

# Tranformed Dataframes

In [159]:
Xtrain.head()

,GoogleMetric:Page Values,Zone,Type of Traffic,WeekendPurchase,1,4,5
0,-0.286256,2.439065,-0.270932,-0.555338,0.0,0.0,0.0
1,-0.286256,-0.482898,-0.022216,-0.555338,0.0,0.0,0.0
2,-0.286256,-0.900322,-0.022216,1.815699,0.0,0.0,0.0
3,-0.286256,-0.065475,-0.270932,-0.555338,0.0,0.0,0.0
4,-0.286256,-0.900322,0.226500,1.815699,0.0,0.0,0.0


In [160]:
Xtest.head()

,GoogleMetric:Page Values,Zone,Type of Traffic,WeekendPurchase,1,4,5
0,-0.286256,-0.900322,-0.768364,-0.555338,0.0,0.0,0.0
1,-0.286256,-0.900322,-0.519648,-0.555338,0.0,0.0,0.0
2,-0.286256,-0.900322,-0.270932,-0.555338,0.0,0.0,0.0
3,-0.286256,-0.482898,-0.270932,-0.555338,0.0,0.0,0.0
4,-0.286256,0.351948,-0.270932,-0.555338,0.0,0.0,0.0


In [161]:
print(ytrain , ytrain.shape)
ytr

[0 0 0 ... 1 1 1] (14731,)


,Made_Purchase
0,False
1,False
2,False
3,False
4,False
...,...
14726,True
14727,True
14728,True
14729,True


<u>Binary classification</u>
---
* Ridge Regression Classifier
* Linear Regression Classifier
* Knn
* Decision tree
* Gassian Naive Bayes

# Prediction Model

In [162]:
clf = KNeighborsClassifier(n_neighbors=50)
clf.fit(Xtrain , ytrain)

KNeighborsClassifier(n_neighbors=50)

In [163]:
ytrain_pred = clf.predict(Xtrain)
print(ytrain_pred.shape)
print('F1 score = ',f1_score(ytrain, ytrain_pred))

(14731,)
F1 score =  0.36776912271740514


In [164]:
C = confusion_matrix(ytrain, ytrain_pred)
pd.DataFrame(C)

,0,1
0,8340,725
1,4226,1440


In [165]:
ytest_pred = clf.predict(Xtest)
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id']=[i for i in range(len(ytest_pred))]
submission["Made_Purchase"]=ytest_pred.astype(bool)
submission.to_csv('submission.csv',index=False)

In [166]:
print(set(submission.iloc[:,1]))
submission

{False, True}


,id,Made_Purchase
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False
...,...,...
6594,6594,False
6595,6595,False
6596,6596,False
6597,6597,False
